# Información de BCRP 

###### A continuación, vamos a extraer información del BCRP usando su api. Se realizará un proceso de automatización en el cual solo se tienen que elegir las variables que se desean obtener y seleccionar la fecha de interés a analizar.

In [1]:
# Códigos para actualizar intervención cambiarias (Operaciones cambiarias del BCRP con las empresas bancarias (millones US$))

# PD37515TD: Al Contado - Compras
# PD37516TD: Al Contado - Ventas
# PD37517TD: Certificados de Depósito Reajustables del BCRP - Emitido
# PD37518TD: Certificados de Depósito Reajustables del BCRP - Vencido
# PD37519TD: Certificados de Depósito Liquidables en Dólares - Emitido
# PD37520TD: Certificados de Depósito Liquidables en Dólares - Vencido
# PD37521TD: SWAP Cambiarios del BCRP Venta - Pactado
# PD37522TD: SWAP Cambiarios del BCRP Venta - Vencido
# PD37523TD: SWAP Cambiarios del BCRP Compra - Pactado
# PD37524TD: SWAP Cambiarios del BCRP Compra - Vencido
# PD37525TD: REPO Sustitución - Ventas Netas

In [5]:
# Importamos las librerías necesarias para el trabajo
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pandas_datareader import data
import requests

In [3]:
# Intervención diaria BCRP
lista=["PD37515TD","PD37516TD","PD37517TD","PD37518TD","PD37519TD","PD37520TD","PD37521TD","PD37522TD","PD37523TD","PD37524TD","PD37525TD"]

In [6]:
# Este es un loop que me permite extraer todas las series de interés contenidas en "lista".
# Creamos primero un contenedor para colocar las series extraídas
d = {}

for serie in lista: 
    # A continuación, colocamos los datos requeridos para el uso del api
    formato = "json"
    fecha_inicio = "2020-10-01"
    fecha_fin = "2022-11-03"
    url = "https://estadisticas.bcrp.gob.pe/estadisticas/series/api/"+serie+"/"+formato+"/"+fecha_inicio+"/"+fecha_fin+"/"
    # Empezamos extrayendo la información en formato json   
    periodos = requests.get(url).json().get("periods")
    # VALORES
    dato = []
    for i in periodos:
        valores_list = i['values']
        for w in valores_list:
            w = float(w)
            dato.append(w)
            d[serie] = pd.DataFrame(dato)
    # FECHAS
    fechas = []
    for i in periodos:
        nombres = i['name']
        fechas.append(nombres)
        df_fechas = pd.DataFrame(fechas)

In [7]:
# EXPORTAR DATOS: Con este código juntaremos todas las variables extraídas.
# Intervención diaria (Operaciones cambiarias del BCRP con las empresas bancarias (millones US$))
data = pd.concat((df_fechas, d['PD37515TD'], d['PD37516TD'], d['PD37517TD'], d['PD37518TD'], d['PD37519TD'], d['PD37520TD'], d['PD37521TD'], d['PD37522TD'], d['PD37523TD'], d['PD37524TD'], d['PD37525TD']), axis=1, ignore_index=True)
data.rename(columns = {0:'fecha', 1:'compras_contado',2:'ventas_contado',3:'deposito_reajus_emitido',4:'deposito_reajus_vencido',5:'deposito_liqu_emitido',6:'deposito_liqu_vencido',7:'swap_venta_pactado',8:'swap_venta_vencido',9:'swap_compra_pactado',10:'swap_compra_vencido',11:'repo_sustitucion'}, inplace = True)
# Exportamos a excel
data.to_excel('Intervención Diaria.xlsx', sheet_name='Monedas', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, inf_rep='inf', freeze_panes=None, storage_options=None)